In [1]:
!gpustat

/bin/sh: gpustat: command not found


In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [7]:
from sep_cons_experiment import SepConsExperiment
from keras import regularizers
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm
import numpy as np
import keras.backend as K

In [8]:
widths = [25] #list(range(1,5)) + list(range(5,26, 5))
depths =  [150]#list(range(90,151, 10))

print('width:', widths)
print('depth:', depths)


width: [25]
depth: [150]


In [ ]:
epochs = 5000
balance = 0.51

param_grid = ParameterGrid({'activation': ['separating_relu_annealing_dropout'], 'lr': [0.01], 'batch_size': [85,],
                           'depth': depths, 'width': widths,
                           'lmda': [0.0001]})

for params in tqdm(param_grid):
    activation, lr, batch_size = params['activation'], params['lr'], params['batch_size']
    depth, width = params['depth'], params['width']
    lmda = params['lmda']
    
    sep = regularizers.l2(lmda)
    sep.name = lmda
    summary_path = 'moons_grid_summaries'
    name = f'moons-grid-zero'
    print(name)
    exp = SepConsExperiment()
    freq = 1
    rate = K.variable(value=0.1, dtype='float32', name='dropout_rate')

    exp.run(name=name, dataset='moons', epochs=epochs,
            optimizer='adam', lr=lr, batch_size=batch_size,
            depth=depth, width=width, activation=activation,  seed=10,
            kernel_size=None,
            memory=0.25,
            layer_parameters={
                'padding': 'same',
                
                'kernel_initializer': 'zeros',
                'bias_initializer': 'zeros',
                'balance': balance,
                
                'separability_regularizer': sep,
                'use_bias': True,
                'axis': [0,-1],
                'rate': rate,
            },
            log_dir=summary_path,
            annealing_dropout_epochs=1000,
            annealing_dropout_rate=0.9
            )


  0%|          | 0/1 [00:00<?, ?it/s]

moons-grid-zero
Instructions for updating:
Colocations handled automatically by placer.
{'self': <sep_cons_experiment.SepConsExperiment object at 0x11cc81400>, 'name': 'moons-grid-zero', 'dataset': 'moons', 'lr': 0.01, 'batch_size': 85, 'epochs': 5000, 'optimizer': 'adam', 'histogram_freq': 0, 'write_grads': False, 'loss': 'crossentropy', 'epoch_start': 0, 'epoch_freq': 0, 'show_on_train': False, 'seed': 10, 'show_input_layer': False, 'show_local_layers': False, 'show_decision_layers': True, 'batch_freq': 0, 'check_weights': False, 'check_gradients': False, 'check_units': None, 'check_optimizer': False, 'write_graph': False, 'check_numerics': False, 'verbose': 0, 'memory': 0.25, 'write_images': False, 'embeddings_freq': None, 'embeddings_layer_names': None, 'extra': None, 'save_config': True, 'store_img': False, 'lr_schedule': None, 'min_lr': 1e-05, 'max_lr': 0.01, 'min_mtm': 0.85, 'max_mtm': 0.95, 'step_size': 1000, 'plot_matrix': False, 'plot_activations': False, 'plot_all_losses': F